<a href="https://colab.research.google.com/github/AnkitAI07/Arithmetic-question-answering/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import transformers
import datasets
from datasets import load_dataset
from datasets import Dataset
from tqdm import tqdm

In [ ]:
!nvidia-smi

Mon Feb 26 17:34:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Assignment6/EnglishDictionary.csv")
df = df.dropna()

In [ ]:
len(df)

176008

In [ ]:
from sklearn.model_selection import train_test_split


df_train, df_val = train_test_split(df, test_size=0.2,random_state=42)

In [ ]:
len(df_train)

140806

In [ ]:
len(df_val)

35202

In [ ]:
dataset = Dataset.from_pandas(df_train)

In [ ]:
dataset

Dataset({
    features: ['Word', 'Count', 'POS', 'Definition', '__index_level_0__'],
    num_rows: 140806
})

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
model.summary()

Model: "tft5_for_conditional_generation_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# !pip install -q datasets
def preprocess_function(examples):
    inputs = examples["Definition"]
    targets = examples["Word"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tokenized_set = dataset.map(preprocess_function, batched=True)
tf_train_set = model.prepare_tf_dataset(
    tokenized_set,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

Map:   0%|          | 0/140806 [00:00<?, ? examples/s]

In [ ]:
tf_train_set

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, None), dtype=tf.int64, name=None))>

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                 patience=5,
                                                 restore_best_weights=True)

In [ ]:
#Added by Ak
dataset_val = Dataset.from_pandas(df_val)
tokenized_set = dataset_val.map(preprocess_function, batched=True)
tf_val_set = model.prepare_tf_dataset(
    tokenized_set,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

Map:   0%|          | 0/35202 [00:00<?, ? examples/s]

In [ ]:
# history=model.fit(tf_train_set,validation_data=tf_val_set,epochs=250,initial_epoch=200)
history=model.fit(tf_train_set,validation_data=tf_val_set,epochs=251,initial_epoch=200)

Epoch 201/251
7059/8800 [=======================>......] - ETA: 9:30 - loss: 2.3096

In [ ]:
#model.save_pretrained("trail_m4")

In [ ]:
#model.save_weights("trailm4")

In [ ]:
model.load_weights("trailm2")

In [ ]:
outputs=model.generate(tokenizer("Road where cars travel quickly without stopping",return_tensors="tf").input_ids,num_beams=10,num_return_sequences=10)
for i in outputs:
    print(tokenizer.decode(i,skip_special_tokens=True))


speedway
lanes
taxiway
expressway
motorway
rushway
highway
freeway
rapidway
highspeed


In [ ]:
test_df=pd.read_excel('TestData.xlsx',header=None)

In [ ]:
#!pip install openpyxl

In [ ]:
test_df

,0
0,traffic regulations
1,acts of violence
2,acoustic spectrum
3,acquisition of a firm
4,adaptation of a text
...,...
107,counter-example
108,borrowing customer
109,botanical garden
110,bottle garden


In [ ]:
def preprocess_function_3(examples):
    inputs = examples["0"]
    targets = examples["0"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
inferenece_dataset = Dataset.from_pandas(test_df)

In [ ]:
if_tf_ds=inferenece_dataset.map(preprocess_function_3, batched=True)

  0%|                                                                                                                                                                                        | 0/1 [00:00<?, ?ba/s]


In [ ]:
if_tf_ds[1]

In [ ]:
if_tf = model.prepare_tf_dataset(
    if_tf_ds,
    batch_size=32,
    collate_fn=data_collator,
    shuffle=False
)

In [ ]:
from tqdm import tqdm
final=[]
num=0
for i in tqdm(if_tf,total=112//32):
    batch=[]
    output=model.generate(i[0]["input_ids"] ,num_beams=10,num_return_sequences=10)
    b=list(tokenizer.batch_decode(output,skip_special_tokens=True))
    batch=[b[i:i+10]for i in range(0,len(b),10)]

    final+=batch
    num+=1
    if num==1000:
        break



  0%|                                                                                                                                                                                       | 0/31 [00:00<?, ?it/s]/home/pratyushg/anaconda3/envs/pytfv2/lib/python3.10/site-packages/transformers/generation_tf_utils.py:1690: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 13%|██████████████████████▌                                                                                                                                                        | 4/31 [00:16<01:53,  4.20s/it]


In [ ]:
pd.DataFrame(final)

,0,1,2,3,4,5,6,7,8,9
0,speeding,traffic,speedway,sternness,stipulation,stipulations,churning,speed,enactment,churnsey
1,robbery,rioting,violence,treason,riot,incendiary,trespass,gangland,terrorism,robberies
2,spectrogram,spectral,spectrophotometry,spectrophotometric,gamma,infra,spectroscope,spectroscopic,spectrophotography,infrared
3,acquisition,takeover,buyout,IPO,divestiture,acquiring,accretion,acquirement,accederation,appropriation
4,analytic,colloquialism,elaboration,reworking,annotating,transliteration,reversion,annotated,anagram,rewrite
...,...,...,...,...,...,...,...,...,...,...
107,counterexample,antipode,counteremphasis,antisample,antipoise,adversity,countersympathetic,countersynthèse,countersymployment,countersymbol
108,direct,empruntor,creditor,emprunter,loaner,bankroll,debtor,lent,borrowers,patronage
109,columbia,archangel,ornithology,arundo,athenaeum,darwin,ornithopod,faro,inglenook,ornithological
110,vinifera,corkscrew,vitis,botanics,viniculture,cannibalize,chard,winery,botanic,coracle


In [ ]:
#pd.DataFrame(final).to_csv("predict_en",index=None)

In [ ]:
import numpy as np
from tqdm import tqdm
import pandas as pd

In [ ]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-en-de"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [ ]:
model.to('cuda')

In [ ]:
pred_df=pd.read_csv("predict_en")

In [ ]:
pred_df

,0,1,2,3,4,5,6,7,8,9
0,speeding,traffic,speedway,sternness,stipulation,stipulations,churning,speed,enactment,churnsey
1,robbery,rioting,violence,treason,riot,incendiary,trespass,gangland,terrorism,robberies
2,spectrogram,spectral,spectrophotometry,spectrophotometric,gamma,infra,spectroscope,spectroscopic,spectrophotography,infrared
3,acquisition,takeover,buyout,IPO,divestiture,acquiring,accretion,acquirement,accederation,appropriation
4,analytic,colloquialism,elaboration,reworking,annotating,transliteration,reversion,annotated,anagram,rewrite
...,...,...,...,...,...,...,...,...,...,...
107,counterexample,antipode,counteremphasis,antisample,antipoise,adversity,countersympathetic,countersynthèse,countersymployment,countersymbol
108,direct,empruntor,creditor,emprunter,loaner,bankroll,debtor,lent,borrowers,patronage
109,columbia,archangel,ornithology,arundo,athenaeum,darwin,ornithopod,faro,inglenook,ornithological
110,vinifera,corkscrew,vitis,botanics,viniculture,cannibalize,chard,winery,botanic,coracle


In [ ]:
pred=[]
for i,row in pred_df.iterrows():
    l=[i for i in row]
    pred+=[l]


In [ ]:
pred

[['speeding',
  'traffic',
  'speedway',
  'sternness',
  'stipulation',
  'stipulations',
  'churning',
  'speed',
  'enactment',
  'churnsey'],
 ['robbery',
  'rioting',
  'violence',
  'treason',
  'riot',
  'incendiary',
  'trespass',
  'gangland',
  'terrorism',
  'robberies'],
 ['spectrogram',
  'spectral',
  'spectrophotometry',
  'spectrophotometric',
  'gamma',
  'infra',
  'spectroscope',
  'spectroscopic',
  'spectrophotography',
  'infrared'],
 ['acquisition',
  'takeover',
  'buyout',
  'IPO',
  'divestiture',
  'acquiring',
  'accretion',
  'acquirement',
  'accederation',
  'appropriation'],
 ['analytic',
  'colloquialism',
  'elaboration',
  'reworking',
  'annotating',
  'transliteration',
  'reversion',
  'annotated',
  'anagram',
  'rewrite'],
 ['surcharge',
  'superannuation',
  'levy',
  'annuity',
  'exorbitant',
  'treble',
  'superannuated',
  'repercussion',
  'excoriation',
  'exempt'],
 ['surcharge',
  'accretion',
  'accretionary',
  'escrow',
  'instalment'

In [ ]:
Final=[]
for i in tqdm(pred):
    input_ids = tokenizer(i,padding=True, return_tensors="pt").input_ids
    output = model.generate(input_ids.to('cuda'))
    batch=list(tokenizer.batch_decode(output,skip_special_tokens=True))
    Final+=[batch]

  0%|          | 0/112 [00:00<?, ?it/s]/home/pratyushg/anaconda3/envs/torch_env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 112/112 [00:27<00:00,  4.09it/s]


In [ ]:
pd.DataFrame(Final)

,0,1,2,3,4,5,6,7,8,9
0,Geschwindigkeitsüberschreitung,Verkehr,Schnellstraße,Strenge,Festlegung,Bestimmungen,Aufwühlen,Geschwindigkeit,Verabschiedung,churnsey
1,Raubüberfall,Ausschreitungen,Gewalt,Landesverrat,Aufruhr,Brandstiftung,Übertretung,Gangland,Terrorismus,Raubüberfälle
2,Spektrogramm,Spektralspektralspektralspektralspektralspektr...,Spektrophotometrie,spektrophotometrisch,Gamma,Infrarot,Spektroskop,spektroskopisch,Spektrofotografie,Infrarot
3,Erwerb,Übernahme,Aufkauf,Börsengang,Veräußerung,Erwerb,Akkumulation,Erwerb,Beitritt,Aneignung
4,analytisch,Umgangssprache,Ausarbeitung,Überarbeitung,Anmerkungen,Transliteration,Rückfall,mit Kommentaren versehen,Anagramm,Umschreiben
...,...,...,...,...,...,...,...,...,...,...
107,Gegenbeispiel,Antipode,Gegengewicht,Antistichprobe,Gegengewicht,Widrigkeiten,Gegensympathisch,Gegensynthese,Gegensymptome,Gegensymbol
108,Direkt,empruntor,Gläubiger,emprunter,Leihgeber,Bankroll,Schuldner,verliehen,Kreditnehmer,Schirmherrschaft
109,Kolumbien,Erzengel,Vogelkunde,Arundo,athenaeum,darwin,Ornithopode,Faro,Unbegreiflichkeit,ornithologisch
110,Weinreben,Korkenzieher,Sehnenentzündung,Botanik,Weinbau,Kannibalisieren,Mangold,Weingut,botanisch,Korakel
